# Skriti Markovi modeli

Model zapišemo s slovarjem slovarjev. Na primer, za metanje kovancev:

![Loaded coin model.](figs/07-loaded-coin.jpg)

In [1]:
T = {"F": {"F": 0.95, "L": 0.05},
     "L": {"F": 0.05, "L": 0.95}}
E = {"F": {"o": 0.5, "-": 0.5},
     "L": {"o": 0.7, "-": 0.3}}
start = "F"

## Generiranje zaporedij

<font color="green"><b>Naredi sam/a.</b></font> Zapiši funkcijo generate_hmm_sequence, ki sprejme skriti Markov model in vrne zaporedje dolžine n (skrito in vidno zaporedje).

Še prej zapišite funkcijo weighted_choice, ki na podlagi uteži (v vrednosti) naključno izbere vrednost (v ključu slovarja).

In [2]:
import random
random.seed(42)

def weighted_choice(weighted_items):
    rnd = random.random() * sum(weighted_items.values())
    for i,v  in weighted_items.items():
        rnd-=v
        if(rnd <0):
            return i

    """Random choice given the list of elements and their weights"""
    


Zdaj pa funkcijo generate_hmm_sequence:

In [3]:
def generate_hmm_sequence(h, T, E, n):
    
    s = weighted_choice(E[h])#vzami verjetnosti vidnih simbolov ko si v stanju h
    yield h,s
    for _ in range(n-1): #preostanek
        h = weighted_choice(T[h])#premakni se v naslednje skrito stanje
        yield h, weighted_choice(E[h])
    
    
    """
    HMM sequence given start state,
    transition, emission matrix and sequence length
    
    return zip(hidden_path, visible_sequence)
    """
    

In [4]:
print(weighted_choice({"F": 0.05, "L": 0.95}))

L


Generiraj nekaj zaporedij različnih dolžin.

In [5]:
list(generate_hmm_sequence('F', T, E, 5))

[('F', 'o'), ('F', 'o'), ('F', '-'), ('F', 'o'), ('F', 'o')]

In [6]:
list(generate_hmm_sequence('F', T, E, 20))

[('F', 'o'),
 ('F', 'o'),
 ('F', '-'),
 ('F', 'o'),
 ('F', '-'),
 ('F', '-'),
 ('F', 'o'),
 ('F', '-'),
 ('F', 'o'),
 ('F', '-'),
 ('F', '-'),
 ('F', '-'),
 ('L', 'o'),
 ('L', '-'),
 ('L', '-'),
 ('L', '-'),
 ('F', 'o'),
 ('F', 'o'),
 ('F', 'o'),
 ('F', '-')]

Model poskusite uporabiti tudi na primeru goljufivega kazinoja:

![Loaded dice model.](figs/07-loaded-dice.jpg)

In [7]:
A = A = ["1", "2", "3", "4", "5", "6"]
E = {"F": {a: 1/6. for a in A},
     "L": {a: 1/10 if a != "6" else 0.5 for a in A}}
T = {"F": {"F": 0.95, "L": 0.05},
     "L": {"F": 0.1, "L": 0.9}}
start = "F"

In [8]:
list(generate_hmm_sequence('F', T, E, 5))

[('F', '3'), ('F', '2'), ('F', '6'), ('F', '4'), ('F', '5')]

In [9]:
list(generate_hmm_sequence('F', T, E, 40))

[('F', '1'),
 ('F', '6'),
 ('F', '4'),
 ('F', '6'),
 ('F', '2'),
 ('F', '2'),
 ('F', '2'),
 ('F', '6'),
 ('F', '4'),
 ('F', '6'),
 ('F', '2'),
 ('F', '4'),
 ('F', '4'),
 ('F', '3'),
 ('F', '6'),
 ('F', '1'),
 ('F', '1'),
 ('F', '5'),
 ('F', '1'),
 ('F', '6'),
 ('F', '6'),
 ('F', '1'),
 ('F', '5'),
 ('F', '2'),
 ('F', '1'),
 ('F', '3'),
 ('L', '6'),
 ('L', '6'),
 ('L', '6'),
 ('L', '3'),
 ('L', '6'),
 ('L', '6'),
 ('L', '6'),
 ('L', '1'),
 ('L', '1'),
 ('L', '6'),
 ('L', '4'),
 ('F', '6'),
 ('F', '1'),
 ('F', '1')]

## Gradnja modela iz podatkov

Napišite funkcijo build_hmm, ki bo sprejela vidno in skrito zaporedje, ter vrnila skriti Markov model, slovarja T in E.

In [30]:
from collections import Counter, defaultdict

def normalize(dic):
    
    s = sum(dic.values())
    return (k, dic[k]/s for k in dic)
def build_hmm(h, x):
    
    
    t = {}
    
    for (i,j), cn in Counter(zip(h, h[1:])).items():
        t.setdefault(i,{}).setdefault(j,cn)
    t = {}
    for i,d in t.items():
        T[i] = normalize(d)
    
    c = Counter(zip(h,x))
    E = {}
    for h in T.keys():
        E[h]= normalize({xi:c((pi,xi)) for pi, xi in c if pi==h})
    
    
    return T, E


SyntaxError: invalid syntax (<ipython-input-30-b61f1052c692>, line 6)

In [11]:
h, x = zip(*list(generate_hmm_sequence('F', T, E, 40)))

In [25]:
build_hmm(h, x)

('F', 'F', 'F', 'F', 'F', 'F', 'F', 'L', 'L', 'L', 'L', 'L', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L')
('5', '1', '4', '6', '2', '5', '2', '6', '6', '3', '6', '3', '4', '6', '1', '5', '1', '4', '5', '2', '3', '3', '5', '1', '4', '3', '5', '3', '6', '6', '1', '6', '6', '2', '3', '1', '6', '6', '5', '6')


TypeError: string indices must be integers